In [ ]:
from pathlib import Path
from Bio.PDB import PDBParser, PDBIO, NeighborSearch
from Bio.PDB import Selection
from utils.docking import sdf2centroid

In [39]:
def extract_res(pdb_file, center:list, threshold=10.0):
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure('retract', pdb_file)

    atoms = Selection.unfold_entities(structure, 'A')
    ns = NeighborSearch(atoms)

    nearby_residues = set()
    
    for atom in ns.search(center, threshold):
        nearby_residues.add(atom.get_parent())

    # Create a new structure with only the nearby residues
    new_structure = structure.copy()
    for model in new_structure:
        for chain in model:
            for residue in list(chain):
                if residue not in nearby_residues:
                    chain.detach_child(residue.id)

    output_file = Path(pdb_file).with_suffix('.pocket10.pdb')
    io = PDBIO()
    io.set_structure(new_structure)
    io.save(str(output_file))
    print(f"Saved {output_file}")

In [40]:
for pok in Path('../../data/clean_targets').glob('*/'):
    pdb = next(pok.glob('*.pdb'))
    print(f"Processing {pdb}")

    # Extract the center from the SDF file
    try:
        sdf = next(pok.glob('*.sdf'))
        center = sdf2centroid(sdf)
    except StopIteration:
        import json
        center = json.loads(
            Path(f"../../Targets/{pok.name}/center.json").read_text()
            )["center"]
    extract_res(pdb, center)

Processing ../../data/clean_targets/NAMPT/NAMPT_7ppe_rec_A.pdb
Saved ../../data/clean_targets/NAMPT/NAMPT_7ppe_rec_A.pocket10.pdb
Processing ../../data/clean_targets/PRMT5/PRMT5_7s1s_rec_A_empty.pdb
Saved ../../data/clean_targets/PRMT5/PRMT5_7s1s_rec_A_empty.pocket10.pdb
Processing ../../data/clean_targets/JAK2/JAK2_8bm2_rec_A.pdb
Saved ../../data/clean_targets/JAK2/JAK2_8bm2_rec_A.pocket10.pdb
Processing ../../data/clean_targets/5HT2A-AF/5HT2A-AF_P28223-F1-model_v4_rec_A.pdb
Saved ../../data/clean_targets/5HT2A-AF/5HT2A-AF_P28223-F1-model_v4_rec_A.pocket10.pdb
Processing ../../data/clean_targets/ROCK1/ROCK1_7jou_rec_A.pdb
Saved ../../data/clean_targets/ROCK1/ROCK1_7jou_rec_A.pocket10.pdb
Processing ../../data/clean_targets/TYK2/TYK2_8tb5_rec_A.pdb
Saved ../../data/clean_targets/TYK2/TYK2_8tb5_rec_A.pocket10.pdb
Processing ../../data/clean_targets/BRD4-holo/BRD4-holo_2oss_rec_A.pdb
Saved ../../data/clean_targets/BRD4-holo/BRD4-holo_2oss_rec_A.pocket10.pdb
Processing ../../data/clean_ta